# Sample Code

## 基礎建設

In [1]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

C:\Users\User\.conda\envs\spacy_env\lib\site-packages\ipykernel\parentpoller.py:109: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  warnings.warn("""Parent poll failed.  If the frontend dies,


## 載入資料

In [2]:
import os 
print(os.getcwd())
metadata = getDF('C:/Users/User/PycharmProjects/data-course-sample/meta_All_Beauty.json.gz')
ratings = pd.read_csv('C:/Users/User/PycharmProjects/data-course-sample/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

C:\Users\User\PycharmProjects\data-course-sample\week_3


In [3]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [4]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

In [5]:
import regex as re
from string import digits

def remove_brand(text):
    res = re.sub(brand_pattern, '', text)
    return res

def find_brand(text):
    res = re.match(brand_pattern, text)
    return res.group(0) if res else ''

def extract_price(cell):
    cell = ''.join(c for c in cell if c in digits)
    return int(cell) if len(cell) > 0 else 0
    

ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')
brand_list = metadata['brand'].value_counts().keys().tolist()
brand_list = list(filter(lambda x:len(x)>2, brand_list))
metadata['text_data'] = metadata.agg(lambda x: f'{x["title"]}, {" ".join(x["description"])}', axis=1)
brand_pattern = re.compile(r"(?:{})".format('|'.join([re.escape(x) for x in brand_list])))
metadata['brand_from_title'] = metadata['title'].apply(find_brand)
# metadata['title_without_brand'] = metadata['title'].apply(remove_brand)
metadata['price'] = metadata['price'].apply(extract_price)
metadata['sub_category'] = metadata['rank'].str.split('in ').str[1]
metadata['sub_category'] = metadata['sub_category'].str.replace(r'&amp;', '', regex=True)
metadata['sub_category'] = metadata['sub_category'].str.replace(r'\(', '', regex=True)
metadata['sub_category'] = metadata['sub_category'].str.replace(r'&', '', regex=True)
# metadata['text_data'] = metadata['text_data'].apply(remove_brand)
import pickle
save = True
# metadata = pickle.load(open('metadata.pickle', 'rb'))
if save:
    pickle.dump(metadata, open('metadata_brand.pickle', 'wb'))

In [6]:
metadata['sub_category'].value_counts()

Beauty  Personal Care        32380
Grocery  Gourmet Food           38
Health  Household               21
Sports  Outdoors                 3
Toys  Games                      3
Baby                             2
Clothing, Shoes  Jewelry         2
Home  Kitchen                    1
Automotive                       1
Tools  Home Improvement          1
Name: sub_category, dtype: int64

In [7]:
metadata.drop(metadata[(metadata['sub_category'] != 'BeautyPersonalCare')].index)
metadata_processed = metadata.drop_duplicates(subset=['title', 'asin'], keep=False)
metadata_processed = metadata_processed.drop(['category', 'fit', 'tech1', 'tech2', 'description', 'feature', 'details', 'imageURL', 'imageURLHighRes', 'similar_item'], axis=1)
metadata_processed[(metadata_processed['brand'][0]==metadata_processed['brand_from_title'])].value_counts()
metadata_processed

,title,also_buy,brand,rank,also_view,main_cat,date,price,asin,text_data,brand_from_title,sub_category
0,Loud 'N Clear&trade; Personal Sound Amplifier,[],idea village,"2,938,573 in Beauty & Personal Care (",[],All Beauty,,0,6546546450,"Loud 'N Clear&trade; Personal Sound Amplifier,...",,Beauty Personal Care
1,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,"872,854 in Beauty & Personal Care (",[],All Beauty,,4499,7178680776,No7 Lift &amp; Luminate Triple Action Serum 50...,No7,Beauty Personal Care
2,No7 Stay Perfect Foundation Cool Vanilla by No7,[],No7,"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]",All Beauty,,2876,7250468162,No7 Stay Perfect Foundation Cool Vanilla by No...,No7,Beauty Personal Care
3,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,"1,870,258 in Beauty & Personal Care (",[],All Beauty,,0,7367905066,Wella Koleston Perfect Hair Colour 44/44 Mediu...,Wella,Beauty Personal Care
4,Lacto Calamine Skin Balance Oil control 120 ml...,[],Pirmal Healthcare,"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...",All Beauty,,1215,7414204790,Lacto Calamine Skin Balance Oil control 120 ml...,,Beauty Personal Care
...,...,...,...,...,...,...,...,...,...,...,...,...
32887,"Barielle Pro Textured Grip Cuticle Nipper, Purple",[],,"2,145,325 in Beauty & Personal Care (",[],All Beauty,,995,B01HIWLLUK,"Barielle Pro Textured Grip Cuticle Nipper, Pur...",,Beauty Personal Care
32888,(Buy 3 Get 1 Free) Salon Perfect Eye Makeup Co...,[],Salon Perfect,"1,639,713 in Beauty & Personal Care (",[],All Beauty,,0,B01HJ1K3YK,(Buy 3 Get 1 Free) Salon Perfect Eye Makeup Co...,,Beauty Personal Care
32889,NOW D-Mannose 500 mg - 120 Veg Capsules (Pack ...,"[B01KON9B4S, B079X3YFXS, B00M79OYS6, B000JN4CR...",,"207,410 in Beauty & Personal Care (","[B01KON9B4S, B000JN4CR0, B071ZHMRHS, B01HJ84TN...",All Beauty,,5563,B01HJ84SGM,NOW D-Mannose 500 mg - 120 Veg Capsules (Pack ...,NOW,Beauty Personal Care
32890,12 White Feather Shuttlecocks Birdies Badminto...,[],GBSTORE,"965,673 in Beauty & Personal Care (",[],All Beauty,,1299,B01HJASD20,12 White Feather Shuttlecocks Birdies Badminto...,,Beauty Personal Care


## 資料切分

In [8]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

## 處理文字資料

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.util import  everygrams
import spacy
# 計算商品用標題所表示的 tfidf 矩陣
df = metadata.drop_duplicates('title')
title_without_brand = df['text_data'].to_list()
nlp = spacy.load('en_core_web_sm', disable = ['ner', 'parser', 'textcat'])
for idx, sent in enumerate(title_without_brand):
    title_without_brand[idx] = re.sub('[0-9.,-_]*', '', sent)
title_without_brand = list(nlp.pipe(title_without_brand))

In [79]:
from nltk.util import ngrams
PRODUCT_WORD={
    'soap':['soap', 'oap'],
    'skin':['skin', 'kin'],
    'hair':['hair', ],
    'cream':['cream', 'ream'],
    'makeup':['makeup', 'akeup'],
    'shampoo':['hampoo', 'shampoo'],
    'nail':['nail', 'ail'],
    'oil':['oil', 'oils', 'ils', 'il'], 
    'brush': ['brush','rush'], 
    'face':['face', 'ace', 'acial', 'facial'],
    'lip':['lip'],
    'fragrance':['ragrance', 'fragrance'],
    'body':['body', 'ody'],
    'vitamin':['itamin', 'vitamin'], 
    'black':['lack', ],
    'natural':['natural', 'atural'],
    'cold':['old', 'cold'],
    'spray':['pray', 'spray'],
    'ring':['ring'],
    'clean':['clean', 'lean', 'cleaning'],
    'nose':['ose', 'nose'],
    'jewelry':['ewelry', 'jewelry'],
    'powder':['owder', 'powder'],
    'razor':['razor', 'azor'],
    'shave':['shave'],
    'acid':['acid'],
    'silver':['ilver', 'silver'],
    'eye':['ye', 'eye'],
    'ear':['ar', 'ear']
    
}

def product_collection(key, product, product_dict):    
    if product_dict.get(key):
        product_dict[key].append(product)
    else:
        product_dict[key] = [product]
    return product_dict

def product_generater(product):
    res = dict()
    for key in product.keys():
        for word in product[key]:
            res[word] = key
    return res
def count_tf(text, tf_dict):
    for word in text:
        tf_dict[word] = tf_dict[word]+1 if tf_dict.get(word) else  1
    return tf_dict

title_without_brand_feature = list()
tf_dict = dict()
nlp.Defaults.stop_words |= {'amp', 'perfect', 'ml', 'air', 'use', 'olor', 'src"httpsimagesnasslimagesamazoncomimageselectronicsdetailpagecomparexgif', 'ize', 'border""td', 'ase', 'quality',
                            'product', 'products', 'ree', 'hite', 'results', 'ingredients', 'unce', 'ink', 'formula', 'omen', 'help', 'quot', 'eauty', 'ith', 'ift', 'look', 'br', ''
                           }
stop_word = nlp.Defaults.stop_words
product_keyword = product_generater(PRODUCT_WORD)
product_dict = dict()
check_idx = set()
stop = False
for idx, sent in enumerate(title_without_brand):
    title = list()
    for word in sent:
#         if word.norm_ == 'ase':
#             print(word.norm_)
#             print(sent)
#             print('='*50)
        if  (not((word.norm_ in stop_word) or (word.is_punct))) and len(word)>=2:
            title.append(word.norm_)
            if product_keyword.get(word.norm_) and (idx not in check_idx):
                product_dict = product_collection(word.norm_, sent, product_dict)
                check_idx.add(idx)

    gram = [' '.join(item) for item in list(ngrams(title, 1))]
    tf_dict = count_tf(gram, tf_dict)
    title_without_brand_feature.append(' '.join(title))
tf_dict = sorted(tf_dict.items(), key=lambda value:value[1], reverse=True)

In [80]:
target_word = ['skin', 'hair', 'ody', 'oil', 'ail', 'kin', 'ream', 'omen', 'natural']
count = 0
for key in product_dict.keys():
    count += len(product_dict[key])
print(f'tag_product: {count}, total: {len(title_without_brand)}')
print(len(tf_dict))
for item in tf_dict[:1500]:
    print(item)

tag_product: 20094, total: 32300
63748
('  ', 7749)
('skin', 7257)
('oz', 5992)
('hair', 4506)
('et', 3477)
('ody', 2873)
('il', 2668)
('lack', 2437)
('ail', 2319)
('oil', 2122)
('kin', 2117)
('ream', 2097)
('natural', 2048)
('   ', 1996)
('color', 1752)
('atural', 1707)
('el', 1695)
('td', 1620)
('ip', 1604)
('body', 1598)
('rush', 1566)
('ace', 1528)
('akeup', 1523)
('old', 1393)
('pray', 1330)
('ew', 1325)
('dry', 1324)
('ye', 1263)
('ar', 1253)
('en', 1232)
('oap', 1174)
('soft', 1101)
('smooth', 1079)
('water', 1072)
('lip', 1071)
('otion', 1050)
('nd', 1037)
('ea', 1021)
('ight', 1009)
('high', 995)
('lue', 945)
('ot', 942)
('oils', 942)
('owder', 939)
('clean', 932)
('fragrance', 930)
('ing', 925)
('ed', 923)
('ade', 923)
('new', 921)
('ag', 918)
('face', 917)
('rt', 909)
('ur', 906)
('ong', 906)
('ose', 903)
('size', 892)
('ilver', 892)
('rofessional', 890)
('rganic', 884)
('best', 880)
('hampoo', 880)
('like', 878)
('    ', 876)
('easy', 874)
('helps', 867)
('time', 866)
('nai

In [ ]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(title_without_brand_feature)

In [ ]:
len(vectorizer.max_features)

In [ ]:
# 計算商品間的相似程度
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(tfidf_matrix)
mapping = pd.Series(df.index,index = df['title'])

# 每個商品回傳 k 個最相近的商品
def recommend_item(item_input, k=2):
    try:
        item_index = mapping[item_input]
        similarity_score = list(enumerate(similarity_matrix[item_index]))
        similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
        similarity_score = similarity_score[:k]
        item_indices = [i[0] for i in similarity_score]
        return (df['asin'].iloc[item_indices].tolist())
    except:
        return []

# 利用使用者購買過的商品產生推薦
def recommend_items(items, k):
    res = []
    for d in items:
        res.extend(recommend_item(d, k))
    return res


In [ ]:
print(df['title'][5:10].to_list())
item_list = recommend_items(df['title'][5:10].to_list(), k=10)
for item in item_list:
    print(df[(df['asin']==item)]['title'].to_list())
    print(df[(df['asin']==item)]['description'])
    print('='*50)

# 產生推薦

In [ ]:
from itertools import combinations
from collections import defaultdict

# header: user_id,item_id,rating,timestamp

def recommender(training_data, users=[], k=10):

    # loading data from dataframe
    # user_to_items dict:
    # {
    #   'user': {
    #       'item': ratings...
    #   }...
    # }
    user_to_items = defaultdict(dict)
    for _, row in training_data.iterrows():
        row = dict(row)
        user = row['reviewerID']
        item = row['asin']
        rating = float(row['overall'])

        user_to_items[user][item] = rating

    print("total users before filtering: ", len(user_to_items))

    # remove obscure user to decrease data size
    # filtering params
    remove_obscure_user = True
    user_rating_threshold = 3
    all_users = list(user_to_items.keys())
    for user in all_users:
        ratings = user_to_items[user]
        if remove_obscure_user and len(ratings) < user_rating_threshold:
            del user_to_items[user]

    print("total users  after filtering: ", len(user_to_items))

    # generate item to user mapping dict
    # {
    #   'item': {
    #       'user': ratings...
    #   }...
    # }
    item_to_users = defaultdict(dict)
    for user, items in user_to_items.items():
        for item, rating in items.items():
            item_to_users[item][user] = rating

    # prepare data of computing user similarity 
    init_sim = lambda: [0 for _ in range(3)]
    factory = lambda: defaultdict(init_sim)
    pre_user_similarity = defaultdict(factory)
    n = len(item_to_users)
    index = 0
    for item, user_ratings in item_to_users.items():
        if len(user_ratings) > 1:
            # print(f"item: {item} have been rated by {len(user_ratings)} users progress: {index}/{n}")
            for user1, user2 in combinations(user_ratings.keys(), 2):
                xy = user_ratings[user1] * user_ratings[user2]
                xx = user_ratings[user1] ** 2
                yy = user_ratings[user2] ** 2
                pre_user_similarity[user1][user2][0] += xy
                pre_user_similarity[user1][user2][1] += xx
                pre_user_similarity[user1][user2][2] += yy

                pre_user_similarity[user2][user1][0] += xy
                pre_user_similarity[user2][user1][1] += xx
                pre_user_similarity[user2][user1][2] += yy
        index += 1

    user_similarity = {}
    for src_user in pre_user_similarity:
        user_similarity_order = []
        for dst_user, val in pre_user_similarity[src_user].items():
            xy = val[0]
            xx = val[1]
            yy = val[2]
            div = ((xx*yy) ** 0.5)
            if div == 0:
                continue
            similarity = xy / div
            if similarity < 0:
                continue
            for i, s in enumerate(user_similarity_order):
                target_similarity = s[1]
                if target_similarity < similarity:
                    user_similarity_order.insert(i, (dst_user, similarity))
                    break
            else:
                user_similarity_order.append((dst_user, similarity))
        user_similarity[src_user] = user_similarity_order

    recommendation = {}
    for user in users:
        if user in user_similarity:
            sim_users = user_similarity[user]
            recommended_items = []
            recommended_items_set = set()
            user_have_rated = set(user_to_items[user])
            stop_recommend = False
            for sim_user, _ in sim_users:
                items_from_sim_user = sorted(list(user_to_items[sim_user].items()), key=lambda item: item[1])
                for item, _ in items_from_sim_user:
                    if item not in user_have_rated and item not in recommended_items_set:
                        recommended_items.append(item)
                        recommended_items_set.add(item)
                    if len(recommended_items) >= k:
                        stop_recommend = True
                        break
                if stop_recommend:
                    break
            recommendation[user] = recommended_items
        else:
            recommendation[user] = []
    return recommendation

ratings_by_user = recommender(ratings_trainings, users)
ratings_by_user

## 結果評估

In [ ]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

evaluate(ratings_testings_by_user, ratings_by_user)

time_list = []
time_list = ['2018-07-01','2018-05-01', '2018-01-01', '2016-01-01']
for time in time_list:
    ratings_by_user = recommender(ratings_trainings, time, users)
    score = evaluate(ratings_testings_by_user, ratings_by_user)
    print(f'{time} score: {score}')